In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
#from src.GenerationAttenuationOptimizer import GenerationAttenuationOptimizer
from pathlib import Path
import subprocess
import os
import tempfile
import shutil
import pandas as pd

In [12]:
cwd = Path.cwd()

In [31]:
conca = 'besos'
contaminant = 'Venlafaxina'
txtinout_path = Path().resolve() / 'data' / 'txtinouts' / f'Txtinout_{conca}'
compound_generator_path = Path().resolve().parent / 'traca' / 'traca'/ 'inputs compound generator'
removal_rate_path = compound_generator_path / 'inputs' / 'atenuacions_generacions.xlsx'
channels_geom_path = cwd / 'data' / 'rivs1' / 'canals_tot_ci.shp'


In [37]:
from src.pollution_utils import generate_pollution_observations, observacions_from_conca
obs = generate_pollution_observations(contaminant)
df = observacions_from_conca(channels_geom_path, obs, conca)

In [33]:


first_observation = df.year.min()
year_end = 2022
year_start = max(first_observation-3, 2000) #3 years warm-up
warmup = max(1, first_observation - year_start)


In [39]:
df

,fecha,estacion,cod_estaci,utm_x,utm_y,variable,unidad_med,valor_alfa,valor,gis_id,conca,year,month,day
26,2022-03-23,P01',P01',434214.9880776412,4596185.98375158,Venlafaxina,ng/L,None,0.000233,87,besos,2022,3,23
27,2022-03-21,P05,P05,469332.0745425668,4620024.819463823,Venlafaxina,ng/L,None,0.000008,135,besos,2022,3,21
30,2022-03-21,P05',P05',470934.1159562504,4621356.884794555,Venlafaxina,ng/L,None,0.000011,135,besos,2022,3,21
31,2022-03-23,P01,P01,435034.02183479257,4598088.718273188,Venlafaxina,ng/L,None,0.000264,88,besos,2022,3,23
34,2022-03-23,P06,P06,433074.16222593,4597072.398023075,Venlafaxina,ng/L,None,0.000213,89,besos,2022,3,23
35,2022-03-23,P06',P06',434105.8951421383,4596147.119800509,Venlafaxina,ng/L,None,0.000175,89,besos,2022,3,23
38,2022-03-21,P10,P10,472561.8776177109,4624802.368005624,Venlafaxina,ng/L,None,0.000007,108,besos,2022,3,21
39,2022-03-21,P10',P10',472856.86008605885,4621895.690410834,Venlafaxina,ng/L,None,0.000012,17,besos,2022,3,21


In [27]:
first_observation

2019

In [5]:

a = GenerationAttenuationOptimizer(
    conca, 
    contaminant, 
    txtinout_folder = txtinout_path,
    removal_rate_path = removal_rate_path,
    compound_generator_path = compound_generator_path,
    channels_geom_path = cwd/'data'/'rivs1'/'canals_tot_ci.shp',
    tmp_path = cwd/'data'/'txtinouts'/'tmp',
    compound_features_path = cwd/'data'/'compound_features.xlsx',
    lod_path = os.path.join('data', 'lod.xlsx'),
    n_gen = 1,
    n_workers = 1,
)

In [6]:
res = a.minimize()

1
1
aaa
aqui entra


In [ ]:
res

{'best_params': {'solub': 198.4392600284095,
  'aq_hlife': 52.034491104988234,
  'aq_volat': 1.2173915570602744e-05,
  'mol_wt': 300.7879648748127,
  'aq_resus': 0.0024036667027493644,
  'aq_settle': 0.6127819433765586,
  'ben_act_dep': 0.37676341902399035,
  'ben_bury': 0.0026088453764971926,
  'ben_hlife': 49.09684326547141,
  'kow': 636.8892525398122},
 'best_error': 0.00022145178278898392}

In [12]:
#random df
import numpy as np
import pandas as pd
df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))
df

,A,B,C,D
0,30,14,96,54
1,29,28,58,17
2,76,8,96,58
3,79,12,58,89
4,91,73,32,19
...,...,...,...,...
95,53,26,67,21
96,64,10,7,67
97,99,9,38,4
98,13,68,32,29


In [13]:
for column in df.columns:
    if df[column].dtype.kind == 'i':
        df[column] = df[column].astype(float)
    print(df[column].dtype.kind)
                

            

f
f
f
f


In [9]:
recall_points_path = compound_generator_path / 'inputs' / 'recall_points.xlsx'
edar_data_path = compound_generator_path / 'inputs' / 'edar_data.xlsx'

In [16]:
recall_points_df = pd.read_excel(recall_points_path)
edars_in_conca = recall_points_df[(recall_points_df['edar_code'].notnull()) & (recall_points_df['conca'] == conca)].edar_code.unique()

In [17]:
edars_in_conca

array(['ES9170460001010E', 'ES9171140003010E', 'ES9171650003010E',
       'ES9171850006010E', 'ES9170190001010E', 'ES9171850001010E'],
      dtype=object)

In [20]:
edar_data_df = pd.read_excel(edar_data_path)
edar_data_df = edar_data_df[edar_data_df['codi_eu'].isin(edars_in_conca)]

,codi_eu,codi_aca,nom_edar,poblacio_sanejada,primari,secundari,terciari
11,ES9171850001010E,DSJFB,BEGUDÀ,234,P,SN,NaN
15,ES9170190001010E,DBES,BESALÚ,2360,P,SN,NaN
32,ES9170460001010E,DCDR,CASTELLFOLLIT DE LA ROCA,1048,P,SC,NaN
68,ES9171140003010E,DOLO,OLOT,36979,P,SP,NaN
109,ES9171650003010E,DSJL,SANT JAUME DE LLIERCA,2104,P,SN,NaN
112,ES9171850006010E,DSJF,SANT JOAN LES FONTS,4151,P,SN,NaN


In [45]:
primari = edar_data_df.primari.dropna().unique()
secundari = edar_data_df.secundari.dropna().unique()
terciari_splitted = [s.split(',') for s in edar_data_df.terciari.dropna()]
terciari = set([tecnologia for tren_tractament in terciari_splitted for tecnologia in tren_tractament])


In [47]:
primari

array(['P'], dtype=object)

In [48]:
secundari

array(['SP', 'SN', 'SC'], dtype=object)

In [49]:
terciari

{'CL', 'OTHER', 'SF', 'UF', 'UV'}

In [59]:
set([*primari, *secundari, *terciari])

{'CL', 'OTHER', 'P', 'SC', 'SF', 'SN', 'SP', 'UF', 'UV'}

In [63]:
a = edar_data_df[edar_data_df['codi_eu'] == 'ES9171850001010E']
a

,codi_eu,codi_aca,nom_edar,poblacio_sanejada,primari,secundari,terciari
11,ES9171850001010E,DSJFB,BEGUDÀ,234,P,SN,NaN


In [69]:
a.loc[11][['primari', 'secundari', 'terciari']]


primari        P
secundari     SN
terciari     NaN
Name: 11, dtype: object